# Download CMAQ gridded-comparison

In [1]:
import requests
import os
import time
from tqdm import tqdm
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import gc

In [7]:
# programatically download CMAQ PM2.5 gridded estimates
# https://www.epa.gov/hesc/rsig-related-downloadable-data-files
years = list(range(2010,2021+1))
if not os.path.exists("CMAQ"):
    os.mkdir("CMAQ")
for year in years:
    print(year)
    if not os.path.exists(f"CMAQ/ds_input_cmaq_pm25_{year}.zip"):
        url = f"https://ofmpub.epa.gov/rsig/rsigserver?data/FAQSD/inputs/ds_input_cmaq_pm25_{year}.zip"
        response = requests.get(url, stream=True)
        with open(f"CMAQ/ds_input_cmaq_pm25_{year}.zip", mode="wb") as file:
            for chunk in response.iter_content(chunk_size=10 * 1024):
                file.write(chunk)
        time.sleep(2)
    

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


Different CMAQ years have different numbers of squares:
- 2010: 459x299
- 2011: 396x246
- 2012: 396x246
- 2013: 459x299
- 2014: 459x299
- 2015: 396x246
- 2016: 396x246
- 2017: 396x246
- 2018: 459x299
- 2019: 459x299
- 2020: 459x299
- 2021: 459x299

This is complete madness and I don't know why they would ever do this, but I'll read in 2011 first, save the list of long/lat pairs, then filter all of the 459x299 years by that list to make them 396x246.

In [2]:
# extract the long/lat pairs for the 396x246 points
if not os.path.exists('CMAQ/squareCoords396x246.csv'):
    cmaq2011 = pd.read_csv("CMAQ/ds_input_cmaq_pm25_2011.zip")
    cmaq2011.drop_duplicates(("Lon", "Lat"))[["Lon", "Lat"]].to_csv('CMAQ/squareCoords396x246.csv', index=False)
    del cmaq2011
    gc.collect()  # Forces a garbage collection cycle

squareCoords396x246 = pd.read_csv('CMAQ/squareCoords396x246.csv')

# make lon lat coord_key to compare against for filtering
squareCoords396x246['coord_key'] = squareCoords396x246['Lon'].round(5).astype(str) + '_' + squareCoords396x246['Lat'].round(5).astype(str)
coord_keys_set = set(squareCoords396x246['coord_key'])

In [3]:
# make grouping matrix for CMAQ, 396 x 246 grid -> 80 x 50 (group squares 5x5, with 5x4 and 4x5 rects at the ends)
rows, cols = 396, 246
side_length = 5

group_matrix = np.zeros((rows, cols), dtype=int)

group_id = 1
for i in range(0, rows, side_length):
    for j in range(0, cols, side_length):
        group_matrix[i:i+side_length, j:j+side_length] = group_id
        group_id += 1

display(group_matrix)
flatGroup = group_matrix.flatten()
flatGroup.shape
del group_matrix

array([[   1,    1,    1, ...,   49,   49,   50],
       [   1,    1,    1, ...,   49,   49,   50],
       [   1,    1,    1, ...,   49,   49,   50],
       ...,
       [3901, 3901, 3901, ..., 3949, 3949, 3950],
       [3901, 3901, 3901, ..., 3949, 3949, 3950],
       [3951, 3951, 3951, ..., 3999, 3999, 4000]])

In [4]:
# Convert DataFrames to GeoDataFrames
def to_gdf(df, lat_col='latitude', lon_col='longitude'):
    return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[lon_col], df[lat_col]), crs="EPSG:4326")

# taken from https://ofmpub.epa.gov/rsig/rsigserver?data/FAQSD/docs/2020_DS_Annual_Report.pdf
# Lambert Conformal Projection with true latitudes at 33 and 45 N, coordinate center 97 W, 40 N, converted by ChatGPT to CRS string
cmaqCRS = (
    "+proj=lcc "
    "+lat_1=33 +lat_2=45 "        # standard parallels
    "+lat_0=40 "                  # latitude of origin
    "+lon_0=-97 "                 # central meridian
    "+x_0=0 +y_0=0 "              # false easting/northing
    "+datum=WGS84 +units=m +no_defs"
)

In [10]:
for year in range(2010, 2021+1):
    # go to next loop iteration if data has already been made
    if os.path.exists(f"CMAQ/summarized/{year}_CMAQ_5x5.csv.gz"):
        continue 
    
    print(year)
    
    print("reading CMAQ")
    # convert 459x299 years into 396x246
    if year in (2010, 2013, 2014, 2018, 2019, 2020, 2021):
        reader = pd.read_csv(f"CMAQ/ds_input_cmaq_pm25_{year}.zip", 
                     chunksize=500_000, 
                     usecols=['Lon', 'Lat', 'Date', 'Conc'], 
                     dtype = {'Date': 'category'})
        
        result_chunks = []
        # 101-ish chunks, not exact
        for chunk in tqdm(reader, total = 101):
            chunk['coord_key'] = chunk['Lon'].round(5).astype(str) + '_' + chunk['Lat'].round(5).astype(str)
            filtered = chunk[chunk['coord_key'].isin(coord_keys_set)]
            result_chunks.append(filtered)
    
        # concat lists together, drop coord_key
        cmaq = pd.concat(result_chunks, ignore_index=True).drop('coord_key', axis=1)
        del result_chunks
        gc.collect()  # Forces a garbage collection cycle
    else:
        cmaq = pd.read_csv(f"CMAQ/ds_input_cmaq_pm25_{year}.zip",
                           usecols=['Lon', 'Lat', 'Date', 'Conc'], 
                           dtype = {'Date': 'category'})
    
    print('done reading')
    # now, we have 396 x 246 table
    
    display(cmaq.shape)
    display(cmaq.columns)

    # duplicate groups for number of unique dates
    groupCol = np.tile(flatGroup,len(cmaq["Date"].unique()))
    
    # add 5x5 groups to cmaq, convert to category
    cmaq['group'] = groupCol
    cmaq['group'] = cmaq['group'].astype('category')

    # summarize the conc for each group with the mean of all points within the group
    cmaqSummarized = cmaq.groupby(['group', 'Date'], as_index=False).mean()
    del cmaq
    gc.collect()  # Forces a garbage collection cycle
    
    # project points to the CMAQ grid specified in the documentation
    cmaqSummarizedgeo = to_gdf(cmaqSummarized, 'Lat', 'Lon').to_crs(cmaqCRS)
    del cmaqSummarized
    gc.collect()  # Forces a garbage collection cycle

    # make the square polygons
    voronoi = cmaqSummarizedgeo.voronoi_polygons()

    # create mask for clipping the voronoi, since the voronoi makes weird polygons near the edges
    # this just makes a convex hull that contains all of the points, then extends it slightly with a buffer
    mask = cmaqSummarizedgeo.drop_duplicates(('Lat', 'Lon')).union_all().convex_hull.buffer(0.05e6)
    mask
    
    if not os.path.exists('CMAQ/summarized'):
        os.mkdir('CMAQ/summarized')
    
    print('saving')
    # remove gross edges from voronoi
    voronoiMask = voronoi.intersection(mask)
    voronoiMask.to_file(f'CMAQ/summarized/{year}_CMAQ_5x5_squares.shp', index=False)  

    # add the cmaq data back into the voronoi using intersection between the cmaq point and the voronoi shapes
    cmaqFinal = gpd.sjoin(cmaqSummarizedgeo, gpd.GeoDataFrame(geometry=voronoi), how = 'left')
    cmaqFinal.to_csv(f"CMAQ/summarized/{year}_CMAQ_5x5.csv.gz")

    # clean up memory
    del cmaqSummarizedgeo
    del voronoiMask
    del cmaqFinal
    del mask
    del voronoi
    gc.collect()  # Forces a garbage collection cycle

2011
reading CMAQ
done reading


(35556840, 4)

Index(['Lon', 'Lat', 'Date', 'Conc'], dtype='object')

/tmp/ipykernel_99325/2814230464.py:44: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cmaqSummarized = cmaq.groupby(['group', 'Date'], as_index=False).mean()


saving
2012
reading CMAQ
done reading


...

done reading


(35556840, 4)

Index(['Lon', 'Lat', 'Date', 'Conc'], dtype='object')

/tmp/ipykernel_99325/2814230464.py:44: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cmaqSummarized = cmaq.groupby(['group', 'Date'], as_index=False).mean()


saving


### Find weather station PM2.5 measurements

In [89]:
# programatically download both federally-regulated and non-federally-regulated daily PM2.5 information
# https://aqs.epa.gov/aqsweb/airdata/download_files.html#Raw
years = list(range(2010,2021+1))
if not os.path.exists("PM2.5"):
    os.mkdir("PM2.5")
for year in years:
    print(year)
    if not os.path.exists(f"PM2.5/daily_88101_{year}.zip"):
        url = f"https://aqs.epa.gov/aqsweb/airdata/daily_88101_{year}.zip"
        response = requests.get(url, stream=True)
        with open(f"PM2.5/daily_88101_{year}.zip", mode="wb") as file:
            for chunk in response.iter_content(chunk_size=10 * 1024):
                file.write(chunk)

    if not os.path.exists(f"PM2.5/daily_88502_{year}.zip"):
        url = f"https://aqs.epa.gov/aqsweb/airdata/daily_88502_{year}.zip"
        response = requests.get(url, stream=True)
        with open(f"PM2.5/daily_88502_{year}.zip", mode="wb") as file:
            for chunk in response.iter_content(chunk_size=10 * 1024):
                file.write(chunk)
        time.sleep(2)
    

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [2]:
# federally-regulated sensors
PM2010_88101 = pd.read_csv('PM2.5/daily_88101_2010.zip')
PM2010_88101.describe()

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code
count,230028.000000,230028.000000,230028.000000,230028.0,230028.000000,230028.000000,230028.000000,230028.000000,230028.000000,230028.000000,230028.000000,230028.000000,179303.000000,230028.000000
mean,25.730224,74.355939,678.073887,88101.0,1.858722,38.567237,-96.375154,5.911054,99.943316,10.020274,12.239993,2.570239,45.534603,147.721843
std,16.174648,87.860608,1552.397756,0.0,1.014308,6.455165,20.827498,9.313001,12.734653,6.677099,11.019930,5.982236,19.544733,23.996032
min,1.000000,1.000000,1.000000,88101.0,1.000000,17.712474,-158.088613,1.000000,4.000000,-5.000000,-4.300000,0.000000,0.000000,116.000000
25%,12.000000,21.000000,6.000000,88101.0,1.000000,35.356615,-112.118210,1.000000,100.000000,5.375000,6.000000,0.000000,31.000000,118.000000
50%,24.000000,57.000000,19.000000,88101.0,1.000000,39.551300,-87.913504,1.000000,100.000000,8.500000,9.700000,0.000000,47.000000,145.000000
75%,40.000000,99.000000,1001.000000,88101.0,3.000000,41.707570,-80.580717,1.000000,100.000000,13.100000,15.500000,0.000000,58.000000,170.000000
max,78.000000,810.000000,9997.000000,88101.0,9.000000,64.845690,-64.784868,24.000000,400.000000,195.000000,886.500000,23.000000,270.000000,236.000000


In [6]:
# non-federally regulated sensors
PM2010_88502 = pd.read_csv('PM2.5/daily_88502_2010.zip')
PM2010_88502.describe()

/tmp/ipykernel_54709/2014225991.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  PM2010_88502 = pd.read_csv('PM2.5/daily_88502_2010.zip')


,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Pollutant Standard,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code
count,358318.000000,358318.000000,358318.000000,358318.0,358318.000000,358318.000000,358318.000000,0.0,358318.000000,358318.000000,358318.000000,358318.000000,358318.000000,195264.000000,358318.000000
mean,30.898409,76.210617,822.854972,88502.0,2.994907,38.536041,-97.792728,NaN,11.278111,99.224122,9.035660,13.082529,15.809669,41.235799,713.797744
std,15.652524,79.793182,1894.886011,0.0,0.868182,6.293628,17.779379,NaN,11.321170,5.422669,6.229058,11.935844,8.941324,20.314123,98.360287
min,1.000000,1.000000,1.000000,88502.0,1.000000,18.334399,-160.508331,NaN,1.000000,4.000000,-7.000000,-7.000000,0.000000,0.000000,118.000000
25%,17.000000,29.000000,7.000000,88502.0,3.000000,33.676490,-117.330980,NaN,1.000000,100.000000,4.682609,6.000000,7.000000,25.000000,702.000000
50%,35.000000,57.000000,22.000000,88502.0,3.000000,39.154000,-94.167468,NaN,1.000000,100.000000,7.700000,10.300000,22.000000,42.000000,715.000000
75%,42.000000,101.000000,1002.000000,88502.0,3.000000,43.123704,-82.619167,NaN,24.000000,100.000000,11.900000,17.000000,23.000000,56.000000,760.000000
max,78.000000,510.000000,9997.000000,88502.0,12.000000,66.930930,-64.795972,NaN,24.000000,100.000000,114.087500,997.000000,23.000000,192.000000,810.000000


### Find weather station wind measurements

In [21]:
# download bulk meteostat data
if not os.path.exists("meteostat/bulk_data_full.json.gz"):
    url = "https://bulk.meteostat.net/v2/stations/full.json.gz"
    response = requests.get(url, stream=True)
    if not os.path.exists("meteostat"):
        os.mkdir("meteostat")
        with open("meteostat/bulk_data_full.json.gz", mode="wb") as file:
            for chunk in response.iter_content(chunk_size=10 * 1024):
                file.write(chunk)


In [22]:
import json
import gzip

with gzip.open('meteostat/bulk_data_full.json.gz', 'r') as fin:
    full = json.loads(fin.read().decode('utf-8'))
usStations = [x for x in full if x["country"] == "US"]
display(len(usStations))
display(usStations[0])

2935

{'id': '04AEH',
 'name': {'en': 'Norwich'},
 'country': 'US',
 'region': 'NY',
 'identifiers': {'national': None, 'wmo': None, 'icao': 'KOIC'},
 'location': {'latitude': 42.5665, 'longitude': -75.5242, 'elevation': 312},
 'timezone': 'America/New_York',
 'inventory': {'model': {'start': '2022-04-24', 'end': '2025-04-23'},
  'hourly': {'start': '2022-04-23', 'end': '2025-04-14'},
  'daily': {'start': '2022-04-23', 'end': '2022-04-26'},
  'monthly': {'start': None, 'end': None},
  'normals': {'start': None, 'end': None}}}

In [23]:
from datetime import datetime

# filter stations to only stations that have at least some data between 2010 and 2021
start_date = datetime.strptime("2010-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2021-12-31", "%Y-%m-%d")
filteredStations = [station for station in usStations if \
 (station['inventory']['daily']['start'] is not None and \
  datetime.strptime(station['inventory']['daily']['start'], "%Y-%m-%d") < end_date \
  and datetime.strptime(station['inventory']['daily']['end'], "%Y-%m-%d") > start_date)]
len(filteredStations)

2303

In [24]:
ids = [x['id'] for x in filteredStations]
ids[0:3]

['69007', '69015', '69019']

In [25]:
if not os.path.exists("meteostat/daily_wind_station_data_2010-2021.csv.gz"):
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    # Import Meteostat library and dependencies
    from datetime import datetime
    from meteostat import Daily
    
    # Set time period
    start = datetime(2010, 1, 1)
    end = datetime(2021, 12, 31)
    
    # Get daily data
    stationData = Daily(ids, start, end)
    stationData = data.fetch()

    # remove rows where no wind was collected
    stationData[~stationData['wdir'].isna()].to_csv("meteostat/daily_wind_station_data_2010-2021.csv.gz")

# if the file already exists, just read from the csv. this also changes the index in the original table
# dtype needs to be specified otherwise the first numeric entries are numbers and the rest are strings. they should all be strings
stationData = pd.read_csv("meteostat/daily_wind_station_data_2010-2021.csv.gz", dtype={'station': 'string'})

# these stations did not have any wind data (or at least i checked the first one)
set(ids) - set(stationData['station'])

{'70179', '70218', '70275', '72378', '72608', '72652', '72789', 'KGVW0'}

In [26]:
stationLocs = pd.DataFrame([(x['id'], x['location']['latitude'], x['location']['longitude']) for x in filteredStations], columns = ('station', 'latitude', 'longitude'))

In [27]:
stationLocs.iloc[0:3]

,station,latitude,longitude
0,69007,36.6815,-121.7617
1,69015,34.2962,-116.1622
2,69019,32.4198,-99.8554


In [28]:
# merge the lat/long locations back into the station data
stationMerged = stationData.merge(stationLocs, on='station', how='left')
display(stationMerged.iloc[0:3])
# check to make sure all stations got the lat/long
display(any(stationMerged['latitude'].isna()))
display(any(stationMerged['longitude'].isna()))

,station,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,latitude,longitude
0,69007,2021-01-02,8.6,5.0,12.0,2.2,NaN,43.0,5.9,NaN,1024.2,NaN,36.6815,-121.7617
1,69007,2021-01-03,10.5,9.0,15.0,1.0,NaN,74.0,5.6,NaN,1024.0,NaN,36.6815,-121.7617
2,69007,2021-01-04,9.7,5.0,15.0,2.2,NaN,173.0,10.2,NaN,1022.2,NaN,36.6815,-121.7617


False

False